In [ ]:
# prep data

# Design model 
# construct loss and optimiser
# Run training loop
    # forward pass - copute prediction
    # compute loss
    # backward pass 
    # update weights using optimiser
    # clear gradients using optimser

In [ ]:
import torch 
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt


# Prepping data for linear regression
X_numpy, Y_numpy = datasets.make_regression(n_samples=100, n_features=1, noise=30, random_state=1)

x = torch.from_numpy(X_numpy.astype(np.float32))
y = torch.from_numpy(Y_numpy.astype(np.float32))
y = y.view(y.shape[0], 1)  # dc

n_samples, n_features = x.shape # 100 ,1

input_size = n_features
output_size = 1

# setup the hyperparams
lr = 0.001
iterations = 1000


# setup the model 
model = nn.Linear(in_features=input_size, out_features=output_size)

loss = nn.MSELoss()

optimiser = torch.optim.SGD(params=model.parameters(),lr=lr)

# training loop

for epoch in range(iterations):
    
    # forward pass
    y_predicted = model(x)
    
    # loss
    lo = loss(y, y_predicted)
    
    # back prop and calulate gradient
    lo.backward()
    
    # update weights
    optimiser.step()
    
    # empty gradients
    optimiser.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f"epoch = {epoch + 1}, loss = {lo.item():.8f}")

predicted = model(x).detach().numpy()
plt.plot(X_numpy, Y_numpy, "ro")
plt.plot(X_numpy, predicted, 'b')
plt.show()



In [ ]:
# logistic regression


import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd


print(pd.__version__)

bc = datasets.load_breast_cancer()
print(list(bc.target_names))
X, y = bc.data, bc.target
n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


# plotting the dataset just for fun

sc = StandardScaler()  # Under stand this part 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))  
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

class LogisticRegression(nn.Module):
    def __init__(self, input_features, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear = nn.Linear(input_features, 1)
    
    def forward(self, x) -> torch.Tensor:
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    

model = LogisticRegression(input_features=n_features)

# 2 Loss and optimiser 
criterion = nn.BCELoss()  # Notice the loss is binary cross entropy loss in this case (read about it)

# Otpimser
learning_rate = 0.01
optimser = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3 training loop 

num_epoch = 100
for epoch in range(num_epoch):
    
    y_predicted = model.forward(X_train)
    
    loss = criterion(y_predicted, y_train)
    
    loss.backward()
    
    # update weight
    
    optimser.step()
    optimser.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f" epoch : {epoch+1}, loss = {loss.item():.4f}")
    


# Now evaluate the model, rememebr it sigmoid activate at the end so ouput will be between 0 and 1

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"accuracy = {acc:.5f}")


# To increase accuracy you can change epoch, learning rate or even a different optimiser
 